In [ ]:
!pip install -Uqq fastbook

     |████████████████████████████████| 727kB 3.4MB/s 
     |████████████████████████████████| 61kB 6.3MB/s 
     |████████████████████████████████| 1.0MB 10.7MB/s 
     |████████████████████████████████| 358kB 23.6MB/s 
     |████████████████████████████████| 51kB 6.2MB/s 
     |████████████████████████████████| 40kB 5.3MB/s 
     |████████████████████████████████| 40kB 5.3MB/s 
     |████████████████████████████████| 92kB 9.5MB/s 
     |████████████████████████████████| 61kB 7.2MB/s 
     |████████████████████████████████| 51kB 6.9MB/s 
     |████████████████████████████████| 2.6MB 28.7MB/s 


## Import required Libraries

In [ ]:
import fastbook
fastbook.setup_book()

Mounted at /content/gdrive


## Download datset

In [ ]:
from fastbook import *

In [ ]:
from fastai.vision.all import *
path = untar_data(URLs.MNIST_TINY)

In [ ]:
dblock = DataBlock(blocks=(ImageBlock(), CategoryBlock()),
                   get_items=get_image_files,
                   get_y=parent_label,
                   item_tfms=Resize(460),
                   batch_tfms=aug_transforms(size=224, min_scale=0.75))
dls = dblock.dataloaders(path, bs=64, num_workers=8)

In [ ]:
dls

In [ ]:
# model = xresnet18()
# learn = cnn_learner(dls, resnet18, metrics=accuracy, pretrained=False) # Learner(dls, model, loss_func=CrossEntropyLossFlat(), metrics=accuracy)
# learn.fit_one_cycle(2, 0.01)

model = xresnet18()
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), metrics=accuracy)
learn.fit_one_cycle(2, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.234801,1.036911,0.529825,06:33
1,1.517848,1.673015,0.529825,06:38


## Normalization

In [ ]:
x,y = dls.one_batch()
x.mean(dim=[0,2,3]),x.std(dim=[0,2,3])

(TensorImage([0.1764, 0.1764, 0.1764]), TensorImage([0.3261, 0.3261, 0.3261]))

In [ ]:
def get_dls(bs, size):
    dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                   get_items=get_image_files,
                   get_y=parent_label,
                   item_tfms=Resize(460),
                   batch_tfms=[*aug_transforms(size=size, min_scale=0.75),
                               Normalize.from_stats(*imagenet_stats)])
    return dblock.dataloaders(path, bs=bs)

In [ ]:
dls = get_dls(64, 224)

In [ ]:
x,y = dls.one_batch()
x.mean(dim=[0,2,3]),x.std(dim=[0,2,3])

(TensorImage([-1.3936, -1.2953, -1.0673]),
 TensorImage([1.3750, 1.4057, 1.3994]))

In [ ]:
model = xresnet18()
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), metrics=accuracy)
learn.fit_one_cycle(2, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.212025,0.844376,0.536842,06:29
1,1.506084,0.431185,0.803509,06:34


## Progressive Resizing

In [ ]:
dls = get_dls(128, 128)
learn = Learner(dls, xresnet18(), loss_func=CrossEntropyLossFlat(), 
                metrics=accuracy)
learn.fit_one_cycle(2, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,2.913307,4.776424,0.522807,05:32
1,1.502870,5.375840,0.452632,05:28


In [ ]:
learn.dls = get_dls(64, 224)
learn.fine_tune(2, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,0.359274,3.154422,0.498246,15:08


epoch,train_loss,valid_loss,accuracy,time
0,0.237345,0.267057,0.950877,15:15
1,0.224604,0.366303,0.859649,15:15


## Test Time Augmentation

In [ ]:
preds,targs = learn.tta()
accuracy(preds, targs).item()

0.8912280797958374

## LabelSmoothing

In [ ]:
model = xresnet18()
learn = Learner(dls, model, loss_func=LabelSmoothingCrossEntropy(), 
                metrics=accuracy)
learn.fit_one_cycle(2, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.391849,2.484337,0.578947,05:32
1,2.556793,4.198382,0.452632,05:28


## Mixup

In [ ]:
model = xresnet50()
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), 
                metrics=accuracy, cbs=MixUp)
learn.fit_one_cycle(5, 3e-3)


epoch,train_loss,valid_loss,accuracy,time
0,3.942723,1.997259,0.522807,05:38
1,2.092185,0.495860,0.831579,05:37
2,1.438345,0.406524,0.877193,05:29
3,1.109406,0.297930,0.915789,05:27
4,0.904362,0.291357,0.912281,05:29
